In [1]:
from __future__ import print_function, division
import sys
sys.path.append('../')
import os
import os.path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
#import matplotlib.pyplot as plt
import time as t
import pandas as pd
from datetime import datetime


# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cv2 # for image manipulation

import torch.utils.data as data
import glob
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json

In [2]:
# Mean and std are as obtained from Autoencoder notebook,
cifar10_mean = [ 0.4732,  0.4637,  0.4293]
cifar10_std = [ 0.2202,  0.2168,  0.2153]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
}

# Define dataloader

In [3]:
class Read_Dataset():
    
    def __init__(self, file_path,transform=None):
        self.data = pd.read_csv(file_path, sep = ',')
        self.label = self.data['class_label'].tolist() 
        self.image = [np.asarray(json.loads(x)) for x in self.data['img_data']]
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        label = self.label[index]
        image_tensor = torch.from_numpy(self.image[index].astype('float32'))
        image = transforms.ToPILImage()(image_tensor)
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

train_file_name = './modified_img_data.csv' 
test_file_name = './modified_img_data_test.csv'

train_val_list = ['train' , 'val']
path_to_input_file = {
    'train' : train_file_name ,
    'val':  test_file_name
    }

In [4]:
start = t.time()
image_datasets = {x: Read_Dataset( file_path = path_to_input_file[x], transform = data_transforms[x])
                  for x in train_val_list}
print("time taken for reading data sets : ", str(t.time() - start))

time taken for reading data sets :  79.38120222091675


In [5]:
batch_size_num = 64
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size_num, shuffle=True, num_workers=4)
              for x in train_val_list}

# Define and train the CNN classifier

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class CNN_classifier(nn.Module):
    def __init__(self):
        # Input size : [batch_size, 3, 32, 32]
        super(CNN_classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, padding = 1) 
        self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 3, padding = 1) 
        self.conv3 = nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size = 3, padding = 1) 
        self.conv3_bn = nn.BatchNorm2d(48)
        self.conv4 = nn.Conv2d(in_channels = 48, out_channels = 48, kernel_size = 3, padding = 1) 
        self.conv4_bn = nn.BatchNorm2d(48)
        self.conv5 = nn.Conv2d(in_channels = 48, out_channels = 48, kernel_size = 3, padding = 1)
        self.conv5_bn = nn.BatchNorm2d(48)
        self.pool = nn.MaxPool2d(2, 2) # reduce the size to half.
        self.conv6 = nn.Conv2d(48, 48, kernel_size = 3, padding = 1)


        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        
        
    # conv->relu->pool->batchnorm
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x)) # Out size : [batch_size, 48, 32, 32]
        x = self.pool(x) # Out size : [batch_size, 48, 16, 16]
        x = self.conv3_bn(x)
        
        
        x = F.relu(self.conv4(x))
        x = self.pool(x) # Out size : [batch_size, 48, 8, 8]
        x = self.conv4_bn(x)
        
        x = F.relu(self.conv5(x))
        x = self.pool(x) # Out size : [batch_size, 48, 4, 4]
        x = self.conv5_bn(x)
        
        # Dropouts
        x = F.dropout(F.relu(self.conv6(x)), p = 0.1)
        x = x.view(-1, 48 * 4 * 4)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


model = CNN_classifier()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.Adadelta(self.model.parameters(), lr=1) #TODO: try with AdaDelta as well.

# Train the classifier network

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN_classifier(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv6): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, o

In [9]:
# Get Validation accuracy while training
def get_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloders['val']:
            images, labels = data
            labels = labels.to(device)
            images = Variable(images.cuda())
#             autoencoder_encodings, _ = autoencoder(images)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    return(100 * correct / total)

In [10]:
max_epoch = 30
running_loss = 0.0
best_epoch_accuracy = 0.0
for epoch in range(max_epoch):
    for i, data in enumerate(dataloders['train']):
        images, labels = data[0].to(device), data[1].to(device)
        images = Variable(images.cuda())

#         autoencoder_encodings, _ = autoencoder(images)

        # zero the parameter gradients
        optimizer.zero_grad()


        # forward + backward + optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
            
    # Print accuracy on validation set after each epoch
    epoch_accuracy = get_accuracy()
    print("epoch_accuracy : ", str(epoch_accuracy))
    # Save best model
    if(epoch_accuracy > best_epoch_accuracy):
        best_epoch_accuracy = epoch_accuracy
        print('Saving best Model...')
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(model.state_dict(), "./weights/vanilla_cnn_classifier.pkl")
        
print('Finished Training')

# if not os.path.exists('./weights'):
#     os.mkdir('./weights')
# torch.save(model.state_dict(), "./weights/cnn_classifier_dropout.pkl")

[1,   100] loss: 1.647
[1,   200] loss: 1.321
[1,   300] loss: 1.144
[1,   400] loss: 1.071
[1,   500] loss: 0.997
[1,   600] loss: 0.943
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.21
Saving best Model...
[2,   100] loss: 1.652
[2,   200] loss: 0.841
[2,   300] loss: 0.814
[2,   400] loss: 0.781
[2,   500] loss: 0.765
[2,   600] loss: 0.737
Accuracy of the network on the 10000 test images: 72 %
epoch_accuracy :  72.86
Saving best Model...
[3,   100] loss: 1.355
[3,   200] loss: 0.705
[3,   300] loss: 0.675
[3,   400] loss: 0.690
[3,   500] loss: 0.662
[3,   600] loss: 0.673
Accuracy of the network on the 10000 test images: 74 %
epoch_accuracy :  74.82
Saving best Model...
[4,   100] loss: 1.197
[4,   200] loss: 0.602
[4,   300] loss: 0.610
[4,   400] loss: 0.620
[4,   500] loss: 0.590
[4,   600] loss: 0.609
Accuracy of the network on the 10000 test images: 75 %
epoch_accuracy :  75.36
Saving best Model...
[5,   100] loss: 1.073
[5,   200] loss: 0.544
[5

# Test the accuracy

In [11]:
# Get Validation accuracy

# Load the best model
path = "./weights/vanilla_cnn_classifier.pkl"
model = CNN_classifier()
state_dict = torch.load(path)
model.load_state_dict(state_dict)
model = model.to(device)
print(model)

correct = 0
total = 0
with torch.no_grad():
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
#         autoencoder_encodings, _ = autoencoder(images)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))


CNN_classifier(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv6): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, o

# Test the class wise accuracy


In [12]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [13]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
#         autoencoder_encodings, _ = autoencoder(images)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(int(labels.shape[0])):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 77 %
Accuracy of   car : 94 %
Accuracy of  bird : 68 %
Accuracy of   cat : 67 %
Accuracy of  deer : 74 %
Accuracy of   dog : 73 %
Accuracy of  frog : 83 %
Accuracy of horse : 85 %
Accuracy of  ship : 88 %
Accuracy of truck : 81 %
